# 유성분검사성적서 전처리

In [ ]:
import pandas as pd
import numpy as np

In [6]:
report = pd.read_csv('유성분검사성적서.csv')
df = report.copy()

/var/folders/xs/4h137c5s21j0snqxv0dhzp4w0000gn/T/ipykernel_7010/104664013.py:1: DtypeWarning: Columns (30,48) have mixed types. Specify dtype option on import or set low_memory=False.
  report = pd.read_csv('유성분검사성적서.csv')


In [7]:
# 불필요 컬럼 drop
drop_col = [
   '혈통번호', '착유일련번호', '체점수(연계)', '유성분검사일련번호', '단백율', '고형율', '유당', '검정일시(ICT)',
'분만나이(연계)', '가수제코드(연계)', '가수제일자(연계)', '분만후첫수정일까지일수', '데이터입력코드', '최근분만일(연계)',
'최종수정일자(연계)', '최종수정횟수(연계)', '최종수정정액코드(연계)', '최근건유일(연계)', '개체명', '농장관리번호'
]

df.drop(columns=drop_col, inplace=True, errors='ignore')


In [8]:
# 불필요 행 drop
drop_row = [
    '개체번호', '비유최고도달일수', '비유후기최고유량',
    '비유초기평균유지율', '비유초기평균단백율', '비유초기평균MNU', '전산차비유지속성'
]

df.dropna(subset=drop_row, inplace=True)


In [9]:
# 305일 관련 행 nan값으로 채우기

nan_305 = ['305일유량', '305일유지량', '305일유단백', '305일무지고형분']

df[nan_305] = df[nan_305].replace(0, np.nan)


In [ ]:
# '비유초기평균MNU' -> '비유초기평균MUN'
df.rename(columns={'비유초기평균MNU': '비유초기평균MUN'}, inplace=True)


In [11]:
# 날짜 기준 정렬
df['검정일자'] = pd.to_datetime(df['검정일자'])
df = df.sort_values(['개체번호', '검정일자'])

# 개체별 전월 유량 가져오기
df['전월유량'] = df.groupby('개체번호')['유량'].shift(1)

# 유량 변화율 계산: ((현재 - 전월) / 전월) * 100
df['유량변화율(%)'] = (df['유량'] - df['전월유량']) / df['전월유량'] * 100

# 질병군 생성: 체세포수 ≥200 & 전월 대비 유량 10% 이상 감소 → 1(의심군), 나머지 2(정상군)
df['질병군'] = np.where((df['체세포수'] >= 200) & (df['유량변화율(%)'] <= -10), 1, 2)

# 중간 계산용 컬럼 제거
df.drop(columns=['전월유량', '유량변화율(%)'], inplace=True)
